## Part 1: Database Setup and Data Modeling

In [1]:
from pymongo import MongoClient
from datetime import datetime, timedelta, UTC
import pandas as pd

# Establish MongoDB connection
client = MongoClient("mongodb://localhost:27017/")

# Create or access the database
db = client["eduhub_db"]

# Create collections with validation rules

# Students collection schema
students_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["student_id", "name", "email", "enrollment_date"],
        "properties": {
            "student_id": {"bsonType": "string", "description": "Unique student ID"},
            "name": {"bsonType": "string", "description": "Full name of the student"},
            "email": {"bsonType": "string", "pattern": "^.+@.+$", "description": "Valid email address"},
            "enrollment_date": {"bsonType": "date", "description": "Date of enrollment"},
            "courses": {
                "bsonType": "array",
                "items": {"bsonType": "string"},
                "description": "List of enrolled course IDs"
            }
        }
    }
}

# Courses collection schema
courses_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["course_id", "title", "instructor"],
        "properties": {
            "course_id": {"bsonType": "string", "description": "Unique course ID"},
            "title": {"bsonType": "string", "description": "Course title"},
            "instructor": {"bsonType": "string", "description": "Instructor name"},
            "credits": {"bsonType": "int", "minimum": 1, "maximum": 10, "description": "Credit hours"}
        }
    }
}

# Instructors collection schema
instructors_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["instructor_id", "name", "email"],
        "properties": {
            "instructor_id": {"bsonType": "string", "description": "Unique instructor ID"},
            "name": {"bsonType": "string", "description": "Instructor full name"},
            "email": {"bsonType": "string", "pattern": "^.+@.+$", "description": "Valid email"},
            "department": {"bsonType": "string", "description": "Department name"}
        }
    }
}


# Drop old collections if exist (to prevent errors during re-run)

db.drop_collection("students")
db.drop_collection("courses")
db.drop_collection("instructors")


# Create collections with validators

db.create_collection("students", validator=students_validator)
db.create_collection("courses", validator=courses_validator)
db.create_collection("instructors", validator=instructors_validator)

print("successfully created Database 'eduhub_db' and collections with validation rules.")


successfully created Database 'eduhub_db' and collections with validation rules.


##Part 2: Data Population

In [ ]:
# Import libraries
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from datetime import datetime, timedelta, UTC
import random

# Connection to MongoDB
MONGO_URI = "mongodb://localhost:27017/"
try:
    client = MongoClient(MONGO_URI)
    # The 'ping' command is to check the connection status
    client.admin.command('ping')
    print("Connection to MongoDB successful! You're ready to go.")
except ConnectionFailure as e:
    print(f"Connection failed: {e}")
    print(f"Please ensure your MongoDB server is running and accessible at {MONGO_URI}")
    exit()

# Get the 'eduhub_db' database and collections from the previous part at the initial insertions
db = client['eduhub_db']
users_collection = db['users']
courses_collection = db['courses']
enrollments_collection = db['enrollments']
lessons_collection = db['lessons']
assignments_collection = db['assignments']
submissions_collection = db['submissions']

print("Database 'eduhub_db' and all collections are set up.")
print("********************************************")


# 2.1 Insert 20 users (15 students, 5 instructors) 
print("Inserting 20 users in progress")

# Sample data contianing names of instructors and studeents
name_pairs = [
    ("Tolu", "Akinola"), ("Chukwudi", "Dike"), ("Femi", "Ojo"), ("Chidi", "Okoye"), ("Emeka", "Nwachukwu"),
    ("Ayomide", "Adewale"), ("Ijeoma", "Okafor"), ("Folake", "Ogunleye"), ("Chioma", "Nwankwo"), ("Zainab", "Umar"),
    ("Halima", "Ibrahim"), ("Aisha", "Bello"), ("Musa", "Abdullahi"), ("Abubakar", "Suleiman"), ("Segun", "Adeyemi"),
    ("Obinna", "Adebayo"), ("Tunde", "Yusuf"), ("Ifeyinwa", "Eke"), ("Sani", "Usman"), ("Bode", "Musa")
]

users_to_insert = []
instructor_ids = []
student_ids = []

for i in range(20):
    role = 'instructor' if i < 5 else 'student'
    
    # Get the name pair from the list using the loop index
    first_name, last_name = name_pairs[i]
    
    user_doc = {
        "userId": f"user_{i+1}",
        "email": f"{first_name.lower()}{last_name.lower()}{i+1}@eduhub.com",
        "firstName": first_name,
        "lastName": last_name,
        "role": role,
        "dateJoined": datetime.now(UTC) - timedelta(days=random.randint(1, 365)),
        "profile": {
            "bio": f"A dedicated {role} on EduHub.",
            "avatar": f"https://example.com/avatars/{i+1}.jpg",
            "skills": ["Python", "MongoDB", "Data Analysis"] if role == 'instructor' else ["Learning"]
        },
        "isActive": True
    }
    users_to_insert.append(user_doc)
    if role == 'instructor':
        instructor_ids.append(user_doc['userId'])
    else:
        student_ids.append(user_doc['userId'])

# Inserting sample data into users collection
users_collection.insert_many(users_to_insert)
print(f" Successfully Inserted {len(users_to_insert)} users.")


#2.2 Insert 8 courses
print("Inserting 8 courses in progress")
courses_to_insert = []
course_ids = []
course_categories = ["Programming", "Design", "Business", "Marketing", "Art", "Science"]
course_titles = [
    "Introduction to Python", "Data Science with Pandas", "UI/UX Design Fundamentals",
    "Digital Marketing Strategies", "Foundations of Art History", "Organic Chemistry",
    "Web Development with MongoDB", "Project Management Basics"
]

for i in range(8):
    course_id = f"course_{i+1}"
    instructor_id = random.choice(instructor_ids)
    course_doc = {
        "course_id": course_id,
        "title": course_titles[i],
        "description": f"A comprehensive course on {course_titles[i]}.",
        "instructor": instructor_id,  #picking a user as an instructor
        "category": random.choice(course_categories),
        "level": random.choice(['beginner', 'intermediate', 'advanced']),
        "duration": random.randint(10, 60),
        "price": random.randint(50, 200),
        "tags": ["online", "2025"],
        "createdAt": datetime.now(UTC),
        "updatedAt": datetime.now(UTC),
        "isPublished": True
    }
    courses_to_insert.append(course_doc)
    course_ids.append(course_id)

courses_collection.insert_many(courses_to_insert)
print(f"{len(courses_to_insert)} courses inserted.")

#2.3 Insert 15 enrollments 
print("Inserting 15 enrollments in progrss")
enrollments_to_insert = []
for i in range(15):
    enrollment_doc = {
        "enrollmentId": f"enrollment_{i+1}",
        "studentId": random.choice(student_ids),  # picking a student user
        "": random.choice(course_ids),    # picking a course
        "enrollmentDate": datetime.now(UTC) - timedelta(days=random.randint(1, 100)),
        "progress": random.uniform(0, 100),
        "status": random.choice(['in-progress', 'completed', 'dropped'])
    }
    enrollments_to_insert.append(enrollment_doc)

enrollments_collection.insert_many(enrollments_to_insert)
print(f"Inserted {len(enrollments_to_insert)} enrollments.")

# 2.4 Insert 25 lessons
print("Inserting 25 lessons in progress")
lessons_to_insert = []
for i in range(25):
    lesson_doc = {
        "lessonId": f"lesson_{i+1}",
        "course_id": random.choice(course_ids),  #picking a course
        "title": f"Lesson {i+1} Title",
        "content": f"Content for lesson {i+1}.",
        "videoUrl": "https://example.com/videos/lesson.mp4",
        "durationMinutes": random.randint(5, 30),
        "order": i + 1,
        "createdAt": datetime.now(UTC)
    }
    lessons_to_insert.append(lesson_doc)

lessons_collection.insert_many(lessons_to_insert)
print(f"Inserted {len(lessons_to_insert)} lessons.")

#2.5 Insert 10 assignments
print("Inserting 10 assignments in progress")
assignments_to_insert = []
assignment_ids = []
for i in range(10):
    assignment_id = f"assignment_{i+1}"
    assignment_doc = {
        "assignmentId": assignment_id,
        "course_id": random.choice(course_ids),  #picking a course
        "title": f"Assignment {i+1} Title",
        "description": f"Description for assignment {i+1}.",
        "dueDate": datetime.now(UTC) + timedelta(days=random.randint(7, 30)),
        "maxScore": 100,
        "createdAt": datetime.now(UTC)
    }
    assignments_to_insert.append(assignment_doc)
    assignment_ids.append(assignment_id)

assignments_collection.insert_many(assignments_to_insert)
print(f"Inserted {len(assignments_to_insert)} assignments.")

# 2.6 Insert 12 assignment submissions
print("Inserting 12 assignment submissions in progress")
submissions_to_insert = []
for i in range(12):
    submission_doc = {
        "submissionId": f"submission_{i+1}",
        "assignmentId": random.choice(assignment_ids),  #picking an assignment
        "studentId": random.choice(student_ids),        # picking a student user
        "submittedAt": datetime.now(UTC) - timedelta(hours=random.randint(1, 24)),
        "submissionUrl": "https://github.com/my-submission",
        "grade": random.randint(50, 100),
        "feedback": "Great work!"
    }
    submissions_to_insert.append(submission_doc)

submissions_collection.insert_many(submissions_to_insert)
print(f"Inserted {len(submissions_to_insert)} assignment submissions.")

print("\n All sample data has been successfully inserted with no errors.")

Connection to MongoDB successful! You're ready to go.
Database 'eduhub_db' and all collections are set up.
********************************************
Inserting 20 users in progress
 Successfully Inserted 20 users.
Inserting 8 courses in progress
8 courses inserted.
Inserting 15 enrollments in progrss
Inserted 15 enrollments.
Inserting 25 lessons in progress
Inserted 25 lessons.
Inserting 10 assignments in progress
Inserted 10 assignments.
Inserting 12 assignment submissions in progress
Inserted 12 assignment submissions.

 All sample data has been successfully inserted with no errors.


C:\Users\hp\AppData\Local\Temp\ipykernel_11084\1108408454.py:120: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "enrollmentDate": datetime.utcnow() - timedelta(days=random.randint(1, 100)),


In [3]:
# Import libraries
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from datetime import datetime, timedelta, UTC
import random

# Connection to MongoDB
MONGO_URI = "mongodb://localhost:27017/"
try:
    client = MongoClient(MONGO_URI)
    # The 'ping' command is to check the connection status
    client.admin.command('ping')
    print("Connection to MongoDB successful! You're ready to go.")
except ConnectionFailure as e:
    print(f"Connection failed: {e}")
    print(f"Please ensure your MongoDB server is running and accessible at {MONGO_URI}")
    exit()

# Get the 'eduhub_db' database and collections from the previous part at the initial insertions
db = client['eduhub_db']
users_collection = db['users']
courses_collection = db['courses']
enrollments_collection = db['enrollments']
lessons_collection = db['lessons']
assignments_collection = db['assignments']
submissions_collection = db['submissions']

print("Database 'eduhub_db' and all collections are set up.")
print("********************************************")


# 2.1 Insert 20 users (15 students, 5 instructors) 
print("Inserting 20 users in progress")

# Sample data contianing names of instructors and studeents
name_pairs = [
    ("Tolu", "Akinola"), ("Chukwudi", "Dike"), ("Femi", "Ojo"), ("Chidi", "Okoye"), ("Emeka", "Nwachukwu"),
    ("Ayomide", "Adewale"), ("Ijeoma", "Okafor"), ("Folake", "Ogunleye"), ("Chioma", "Nwankwo"), ("Zainab", "Umar"),
    ("Halima", "Ibrahim"), ("Aisha", "Bello"), ("Musa", "Abdullahi"), ("Abubakar", "Suleiman"), ("Segun", "Adeyemi"),
    ("Obinna", "Adebayo"), ("Tunde", "Yusuf"), ("Ifeyinwa", "Eke"), ("Sani", "Usman"), ("Bode", "Musa")
]

users_to_insert = []
instructor_ids = []
student_ids = []

for i in range(20):
    role = 'instructor' if i < 5 else 'student'
    
    # Get the name pair from the list using the loop index
    first_name, last_name = name_pairs[i]
    
    user_doc = {
        "userId": f"user_{i+1}",
        "email": f"{first_name.lower()}{last_name.lower()}{i+1}@eduhub.com",
        "firstName": first_name,
        "lastName": last_name,
        "role": role,
        "dateJoined": datetime.now(UTC) - timedelta(days=random.randint(1, 365)),
        "profile": {
            "bio": f"A dedicated {role} on EduHub.",
            "avatar": f"https://example.com/avatars/{i+1}.jpg",
            "skills": ["Python", "MongoDB", "Data Analysis"] if role == 'instructor' else ["Learning"]
        },
        "isActive": True
    }
    users_to_insert.append(user_doc)
    if role == 'instructor':
        instructor_ids.append(user_doc['userId'])
    else:
        student_ids.append(user_doc['userId'])

# Inserting sample data into users collection
users_collection.insert_many(users_to_insert)
print(f" Successfully Inserted {len(users_to_insert)} users.")


#2.2 Insert 8 courses
print("Inserting 8 courses in progress")
courses_to_insert = []
course_ids = []
course_categories = ["Programming", "Design", "Business", "Marketing", "Art", "Science"]
course_titles = [
    "Introduction to Python", "Data Science with Pandas", "UI/UX Design Fundamentals",
    "Digital Marketing Strategies", "Foundations of Art History", "Organic Chemistry",
    "Web Development with MongoDB", "Project Management Basics"
]

for i in range(8):
    course_id = f"course_{i+1}"
    instructor_id = random.choice(instructor_ids)
    course_doc = {
        "course_id": course_id,
        "title": course_titles[i],
        "description": f"A comprehensive course on {course_titles[i]}.",
        "instructor": instructor_id,  #picking a user as an instructor
        "category": random.choice(course_categories),
        "level": random.choice(['beginner', 'intermediate', 'advanced']),
        "duration": random.randint(10, 60),
        "price": random.randint(50, 200),
        "tags": ["online", "2025"],
        "createdAt": datetime.now(UTC),
        "updatedAt": datetime.now(UTC),
        "isPublished": True
    }
    courses_to_insert.append(course_doc)
    course_ids.append(course_id)

courses_collection.insert_many(courses_to_insert)
print(f"{len(courses_to_insert)} courses inserted.")

#2.3 Insert 15 enrollments 
print("Inserting 15 enrollments in progrss")
enrollments_to_insert = []
for i in range(15):
    enrollment_doc = {
        "enrollmentId": f"enrollment_{i+1}",
        "studentId": random.choice(student_ids),  # picking a student user
        "": random.choice(course_ids),    # picking a course
        "enrollmentDate": datetime.now(UTC) - timedelta(days=random.randint(1, 100)),
        "progress": random.uniform(0, 100),
        "status": random.choice(['in-progress', 'completed', 'dropped'])
    }
    enrollments_to_insert.append(enrollment_doc)

enrollments_collection.insert_many(enrollments_to_insert)
print(f"Inserted {len(enrollments_to_insert)} enrollments.")

# 2.4 Insert 25 lessons
print("Inserting 25 lessons in progress")
lessons_to_insert = []
for i in range(25):
    lesson_doc = {
        "lessonId": f"lesson_{i+1}",
        "course_id": random.choice(course_ids),  #picking a course
        "title": f"Lesson {i+1} Title",
        "content": f"Content for lesson {i+1}.",
        "videoUrl": "https://example.com/videos/lesson.mp4",
        "durationMinutes": random.randint(5, 30),
        "order": i + 1,
        "createdAt": datetime.now(UTC)
    }
    lessons_to_insert.append(lesson_doc)

lessons_collection.insert_many(lessons_to_insert)
print(f"Inserted {len(lessons_to_insert)} lessons.")

#2.5 Insert 10 assignments
print("Inserting 10 assignments in progress")
assignments_to_insert = []
assignment_ids = []
for i in range(10):
    assignment_id = f"assignment_{i+1}"
    assignment_doc = {
        "assignmentId": assignment_id,
        "course_id": random.choice(course_ids),  #picking a course
        "title": f"Assignment {i+1} Title",
        "description": f"Description for assignment {i+1}.",
        "dueDate": datetime.now(UTC) + timedelta(days=random.randint(7, 30)),
        "maxScore": 100,
        "createdAt": datetime.now(UTC)
    }
    assignments_to_insert.append(assignment_doc)
    assignment_ids.append(assignment_id)

assignments_collection.insert_many(assignments_to_insert)
print(f"Inserted {len(assignments_to_insert)} assignments.")

# 2.6 Insert 12 assignment submissions
print("Inserting 12 assignment submissions in progress")
submissions_to_insert = []
for i in range(12):
    submission_doc = {
        "submissionId": f"submission_{i+1}",
        "assignmentId": random.choice(assignment_ids),  #picking an assignment
        "studentId": random.choice(student_ids),        # picking a student user
        "submittedAt": datetime.now(UTC) - timedelta(hours=random.randint(1, 24)),
        "submissionUrl": "https://github.com/my-submission",
        "grade": random.randint(50, 100),
        "feedback": "Great work!"
    }
    submissions_to_insert.append(submission_doc)

submissions_collection.insert_many(submissions_to_insert)
print(f"Inserted {len(submissions_to_insert)} assignment submissions.")

print("\n All sample data has been successfully inserted with no errors.")

Connection to MongoDB successful! You're ready to go.
Database 'eduhub_db' and all collections are set up.
********************************************
Inserting 20 users in progress
 Successfully Inserted 20 users.
Inserting 8 courses in progress
8 courses inserted.
Inserting 15 enrollments in progrss
Inserted 15 enrollments.
Inserting 25 lessons in progress
Inserted 25 lessons.
Inserting 10 assignments in progress
Inserted 10 assignments.
Inserting 12 assignment submissions in progress
Inserted 12 assignment submissions.

 All sample data has been successfully inserted with no errors.


In [4]:
# Import libraries
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from datetime import datetime, timedelta, UTC
import random

# Connection to MongoDB
MONGO_URI = "mongodb://localhost:27017/"
try:
    client = MongoClient(MONGO_URI)
    # The 'ping' command is to check the connection status
    client.admin.command('ping')
    print("Connection to MongoDB successful! You're ready to go.")
except ConnectionFailure as e:
    print(f"Connection failed: {e}")
    print(f"Please ensure your MongoDB server is running and accessible at {MONGO_URI}")
    exit()

# Get the 'eduhub_db' database and collections from the previous part at the initial insertions
db = client['eduhub_db']
users_collection = db['users']
courses_collection = db['courses']
enrollments_collection = db['enrollments']
lessons_collection = db['lessons']
assignments_collection = db['assignments']
submissions_collection = db['submissions']

print("Database 'eduhub_db' and all collections are set up.")
print("********************************************")


# 2.1 Insert 20 users (15 students, 5 instructors) 
print("Inserting 20 users in progress")

# Sample data contianing names of instructors and studeents
name_pairs = [
    ("Tolu", "Akinola"), ("Chukwudi", "Dike"), ("Femi", "Ojo"), ("Chidi", "Okoye"), ("Emeka", "Nwachukwu"),
    ("Ayomide", "Adewale"), ("Ijeoma", "Okafor"), ("Folake", "Ogunleye"), ("Chioma", "Nwankwo"), ("Zainab", "Umar"),
    ("Halima", "Ibrahim"), ("Aisha", "Bello"), ("Musa", "Abdullahi"), ("Abubakar", "Suleiman"), ("Segun", "Adeyemi"),
    ("Obinna", "Adebayo"), ("Tunde", "Yusuf"), ("Ifeyinwa", "Eke"), ("Sani", "Usman"), ("Bode", "Musa")
]

users_to_insert = []
instructor_ids = []
student_ids = []

for i in range(20):
    role = 'instructor' if i < 5 else 'student'
    
    # Get the name pair from the list using the loop index
    first_name, last_name = name_pairs[i]
    
    user_doc = {
        "userId": f"user_{i+1}",
        "email": f"{first_name.lower()}{last_name.lower()}{i+1}@eduhub.com",
        "firstName": first_name,
        "lastName": last_name,
        "role": role,
        "dateJoined": datetime.now(UTC) - timedelta(days=random.randint(1, 365)),
        "profile": {
            "bio": f"A dedicated {role} on EduHub.",
            "avatar": f"https://example.com/avatars/{i+1}.jpg",
            "skills": ["Python", "MongoDB", "Data Analysis"] if role == 'instructor' else ["Learning"]
        },
        "isActive": True
    }
    users_to_insert.append(user_doc)
    if role == 'instructor':
        instructor_ids.append(user_doc['userId'])
    else:
        student_ids.append(user_doc['userId'])

# Inserting sample data into users collection
users_collection.insert_many(users_to_insert)
print(f" Successfully Inserted {len(users_to_insert)} users.")


#2.2 Insert 8 courses
print("Inserting 8 courses in progress")
courses_to_insert = []
course_ids = []
course_categories = ["Programming", "Design", "Business", "Marketing", "Art", "Science"]
course_titles = [
    "Introduction to Python", "Data Science with Pandas", "UI/UX Design Fundamentals",
    "Digital Marketing Strategies", "Foundations of Art History", "Organic Chemistry",
    "Web Development with MongoDB", "Project Management Basics"
]

for i in range(8):
    course_id = f"course_{i+1}"
    instructor_id = random.choice(instructor_ids)
    course_doc = {
        "course_id": course_id,
        "title": course_titles[i],
        "description": f"A comprehensive course on {course_titles[i]}.",
        "instructor": instructor_id,  #picking a user as an instructor
        "category": random.choice(course_categories),
        "level": random.choice(['beginner', 'intermediate', 'advanced']),
        "duration": random.randint(10, 60),
        "price": random.randint(50, 200),
        "tags": ["online", "2025"],
        "createdAt": datetime.now(UTC),
        "updatedAt": datetime.now(UTC),
        "isPublished": True
    }
    courses_to_insert.append(course_doc)
    course_ids.append(course_id)

courses_collection.insert_many(courses_to_insert)
print(f"{len(courses_to_insert)} courses inserted.")

#2.3 Insert 15 enrollments 
print("Inserting 15 enrollments in progrss")
enrollments_to_insert = []
for i in range(15):
    enrollment_doc = {
        "enrollmentId": f"enrollment_{i+1}",
        "studentId": random.choice(student_ids),  # picking a student user
        "": random.choice(course_ids),    # picking a course
        "enrollmentDate": datetime.now(UTC) - timedelta(days=random.randint(1, 100)),
        "progress": random.uniform(0, 100),
        "status": random.choice(['in-progress', 'completed', 'dropped'])
    }
    enrollments_to_insert.append(enrollment_doc)

enrollments_collection.insert_many(enrollments_to_insert)
print(f"Inserted {len(enrollments_to_insert)} enrollments.")

# 2.4 Insert 25 lessons
print("Inserting 25 lessons in progress")
lessons_to_insert = []
for i in range(25):
    lesson_doc = {
        "lessonId": f"lesson_{i+1}",
        "course_id": random.choice(course_ids),  #picking a course
        "title": f"Lesson {i+1} Title",
        "content": f"Content for lesson {i+1}.",
        "videoUrl": "https://example.com/videos/lesson.mp4",
        "durationMinutes": random.randint(5, 30),
        "order": i + 1,
        "createdAt": datetime.now(UTC)
    }
    lessons_to_insert.append(lesson_doc)

lessons_collection.insert_many(lessons_to_insert)
print(f"Inserted {len(lessons_to_insert)} lessons.")

#2.5 Insert 10 assignments
print("Inserting 10 assignments in progress")
assignments_to_insert = []
assignment_ids = []
for i in range(10):
    assignment_id = f"assignment_{i+1}"
    assignment_doc = {
        "assignmentId": assignment_id,
        "course_id": random.choice(course_ids),  #picking a course
        "title": f"Assignment {i+1} Title",
        "description": f"Description for assignment {i+1}.",
        "dueDate": datetime.now(UTC) + timedelta(days=random.randint(7, 30)),
        "maxScore": 100,
        "createdAt": datetime.now(UTC)
    }
    assignments_to_insert.append(assignment_doc)
    assignment_ids.append(assignment_id)

assignments_collection.insert_many(assignments_to_insert)
print(f"Inserted {len(assignments_to_insert)} assignments.")

# 2.6 Insert 12 assignment submissions
print("Inserting 12 assignment submissions in progress")
submissions_to_insert = []
for i in range(12):
    submission_doc = {
        "submissionId": f"submission_{i+1}",
        "assignmentId": random.choice(assignment_ids),  #picking an assignment
        "studentId": random.choice(student_ids),        # picking a student user
        "submittedAt": datetime.now(UTC) - timedelta(hours=random.randint(1, 24)),
        "submissionUrl": "https://github.com/my-submission",
        "grade": random.randint(50, 100),
        "feedback": "Great work!"
    }
    submissions_to_insert.append(submission_doc)

submissions_collection.insert_many(submissions_to_insert)
print(f"Inserted {len(submissions_to_insert)} assignment submissions.")

print("\n All sample data has been successfully inserted with no errors.")

Connection to MongoDB successful! You're ready to go.
Database 'eduhub_db' and all collections are set up.
********************************************
Inserting 20 users in progress
 Successfully Inserted 20 users.
Inserting 8 courses in progress
8 courses inserted.
Inserting 15 enrollments in progrss
Inserted 15 enrollments.
Inserting 25 lessons in progress
Inserted 25 lessons.
Inserting 10 assignments in progress
Inserted 10 assignments.
Inserting 12 assignment submissions in progress
Inserted 12 assignment submissions.

 All sample data has been successfully inserted with no errors.
